In [1]:
import os
import sys
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import math
from collections import Counter, defaultdict
d = 0.75

In [2]:
def read_data(PATH):
    
    all_data = []
    for path, dirs, files in os.walk(PATH):
        temp = []
        
        for filename in files:
            filePath = path + '/' + filename
            #print(filePath)
            f = open( filePath, 'r').read()
            temp.append(parse_article(f))

        if temp:
            all_data.append(temp)
    return all_data

In [3]:
def parse_article(f):
#     TODO: tokenize sentence and transform to lower and padding
    article = sent_tokenize(f)
    for sen in article:
        sen = sen.lower()
    return article


In [4]:
def kn_model(uni, bi ,label, str1, str2):
#     TODO: Bigram() + Pcontination()
# KN( wi | wi−1 ) = log( Bigram + λ(wi−1)* PCONTINUATION)
# Bigram : max( C(wi-1, wi) -d )/∑C(wi−1, w*), 0 )
# PCONTINUATION: max( ∑type(w* ,wi)- d )/∑type(wj-1 ,wj) , 0 )
# λ(wi−1) = (d / ∑C(wi−1, w*))*∑type(wi−1 ,w*)
    temp_str1 = str1.lower()
    temp_str2 = str2.lower()

    bigram_word = temp_str1 +' '+temp_str2
    bigram_word_num = bi[label][bigram_word]
    temp_len = len(temp_str1)
    prefix = 0
    prefix__type_num = 0
    for key in bi[label]:
        if key[0:temp_len] == temp_str1:
            prefix += bi[label][key]
            prefix__type_num += 1
    
    bigram=0
    if prefix!=0:
        bigram = max(bigram_word_num-d, 0)/prefix

    
    temp_len2 = len(temp_str2)
    type_of_postfix = 0
    
    if uni[label][temp_str2]!=0:
        for key in bi[label]:
            if key[-temp_len2:] == temp_str2:
                type_of_postfix +=1
    
    p_con = max((type_of_postfix-d)/len(bi[label]), 0)

    lam=0
    if prefix != 0:
        lam = (d/prefix)*prefix__type_num

    kn=-10
    if bigram==0 and lam*p_con==0:
        kn=-10
    else:
        kn = math.log(bigram + (lam*p_con))
    return round(kn,5)

In [5]:
def padding(l):
    for w in l:
        w = w.lower()
    l.insert(0,'<s>')
    l.append("<\s>")
    return l

In [6]:
def counter(l):
    return Counter(l)

In [7]:
def count_2gram(l):
    temp = []
    for idx, val in enumerate(l):
        if idx+1 < len(l):
            temp.append(val +' '+ l[idx+1])
    return Counter(temp)

In [8]:
def score_article():
#     TODO: get label that has highest scores
    return


In [9]:
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s \r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [25]:
if __name__ == '__main__':
    
    
    print("load and preprocess training data")
    train_PATH = '/Users/7ckung/Downloads/reuters/r_train'
    
    all_training_data = read_data(train_PATH)
    
    print('start training')
#     TODO: build kn model
    uni = []
    bi = []
    label_num = 0
    for label in all_training_data:
        uni.append(Counter())
        bi.append(Counter())
        for file in label:
            for line in file:
                temp_line = word_tokenize(line)
                temp_line = padding(temp_line)
                uni[label_num] += Counter(temp_line)
                temp_bi_count = count_2gram(temp_line)
                bi[label_num] += temp_bi_count
        label_num +=1        
    uni_g = []       
    bi_g = []
    for i in range(12):
        uni_g.append(defaultdict(int,uni[i]))
        bi_g.append(defaultdict(int, bi[i]))
        
    kn = []
    label_num = 0
    c=0
    train_data_len = sum(len(t) for t in all_training_data)
    for label in all_training_data:
        kn.append({})
        for file in label:
            for line in file:
                temp_line = word_tokenize(line)
                temp = padding(temp_line)
                for i in range(len(temp)-1):
                    bi_str = temp[i]+ ' ' + temp[i+1]
                    if bi_str not in kn[label_num]:
                        kn[label_num][bi_str] = kn_model(uni_g, bi_g, label_num, temp[i], temp[i+1])
            c+=1
            progress(c,train_data_len)
        label_num +=1    

load and preprocess training data
start training


In [29]:
import json
with open('kn.json', 'w') as outfile:
    json.dump(kn, outfile)

In [35]:
kn = json.loads(open('kn.json').read())

In [36]:
print("start testing")
test_PATH = '/Users/7ckung/Downloads/reuters/r_test'
all_testing_data = read_data(test_PATH)
test_data_len = sum(len(t) for t in all_testing_data)
total_hit = 0
label_seq = 0
dict_count = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0,'10':0,'11':0}
c=0
for label in all_testing_data:
    for file in label:
        predict = [0.00000]*12                    
        result = 0
        #
#         for model_label in range(12):
#             predict_file = 0
        for line in file:
            temp_line = word_tokenize(line)
            temp = padding(temp_line)
            for i in range(len(temp)-1):
                for model_label in range(12):
                    temp_str = temp[i]+ ' ' + temp[i+1]
                    if temp_str not in kn[model_label]:
                        predict[model_label] = predict[model_label] + -10
                    else:
                        predict[model_label] = predict[model_label] + kn[model_label][temp_str]
        result = predict.index(max(predict))
#         print(predict)
        if result == label_seq:
            total_hit +=1
        dict_count[str(result)]+=1
        c+=1
#         ite+=1 
        progress(c,test_data_len)
#         if ite ==2:
#             break
    label_seq +=1
    print('testing category_'+str(label_seq))
print(dict_count)
print('Accuracy: ', total_hit/test_data_len)

start testing
testing category_1-------------------------------------------] 7.3% ... 
testing category_2-------------------------------------------] 10.5% ... 
testing category_3-------------------------------------------] 18.6% ... 
testing category_4-------------------------------------------] 22.7% ... 
testing category_5=====--------------------------------------] 36.6% ... 
testing category_6==========---------------------------------] 45.8% ... 
testing category_7================---------------------------] 54.5% ... 
testing category_8===================------------------------] 59.3% ... 
testing category_9=====================----------------------] 64.1% ... 
testing category_10=========================-----------------] 71.3% ... 
testing category_11==================================--------] 85.9% ... 
testing category_12==========================================] 100.0% ... 
{'0': 15, '1': 0, '2': 73, '3': 4, '4': 90, '5': 99, '6': 138, '7': 13, '8': 5, '9': 3, '10': 127,